# Projeto: Problema de classificação Fashion MNIST
<hr>

#### Explorando dados usando datasets do Keras<br>

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.image import resize

from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

> #### Carregando dados do dataset

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
print(X_train.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28)


In [3]:
vgg16 = VGG16(input_shape=(32, 32, 3), classes=10, weights=None)
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │       2,101,248 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 33,638,218 (128.32 MB)

 Trainable params: 33,638,218 (128.32 MB)

 Non-trainable params: 0 (0.00 B)

> #### Redimensionando as imagens conforme instrução do enunciado

In [4]:
print(X_train.shape)
X_train_1 = np.expand_dims(X_train, axis=-1)
print(X_train_1.shape)
X_train_1 = np.repeat(X_train_1, 3, axis=-1)
print(X_train_1.shape)
X_train_resize = resize(X_train_1, [32, 32])
print(X_train_resize.shape)

(60000, 28, 28)
(60000, 28, 28, 1)
(60000, 28, 28, 3)
(60000, 32, 32, 3)


In [5]:
print(X_test.shape)
X_test_1 = np.expand_dims(X_test, axis=-1)
print(X_test_1.shape)
X_test_1 = np.repeat(X_test_1, 3, axis=-1)
print(X_test_1.shape)
X_test_resize = resize(X_test_1, [32, 32])
print(X_test_resize.shape)

(10000, 28, 28)
(10000, 28, 28, 1)
(10000, 28, 28, 3)
(10000, 32, 32, 3)


In [7]:
y_train_h  = to_categorical(y_train)
y_test_h = to_categorical(y_test)
num_classes = y_test_h.shape[1]

In [8]:
num_pixels = X_train_resize.shape[1] * X_train_resize.shape[2]


In [9]:
vgg16.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history2=vgg16.fit(X_train_resize/255, y_train_h,
                   validation_data=(X_test_resize/255, y_test_h),
          batch_size=128,
          epochs=10,
          verbose=1)
scores = vgg16.evaluate(X_test_resize, y_test_h, verbose=0)
print("%s: %.2f%%" % (vgg16.metrics_names[1], scores[1]*100))

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 60s 99ms/step - accuracy: 0.0994 - loss: 2.3272 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 62s 77ms/step - accuracy: 0.1020 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 40s 75ms/step - accuracy: 0.0992 - loss: 2.3026 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 76ms/step - accuracy: 0.0993 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 42s 78ms/step - accuracy: 0.0975 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - accuracy: 0.0976 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - accuracy: 0.1001 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 41s 78ms/step - accuracy: 0.1002 - loss: 2.3026 - 

In [10]:
# NOVA REDE
(X_train_rede, y_train_rede), (X_test_rede, y_test_rede) = fashion_mnist.load_data()
num_pixels = X_train_rede.shape[1] * X_train_rede.shape[2]
print(X_train_rede.shape)

X_train_rede2 = X_train_rede.reshape(X_train_rede.shape[0], num_pixels)

print(X_train_rede2[0][333])
print(X_train_rede2.shape)

X_test_rede2 = X_test_rede.reshape(X_test_rede.shape[0], num_pixels).astype('float32')

y_train_rede_h = to_categorical(y_train_rede)
y_test_rede_h = to_categorical(y_test_rede)

X_train_rede2 = X_train_rede2/255
X_test_rede2 = X_test_rede2/255

model = Sequential()

model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
model.add(Dense(1024, kernel_initializer='normal', activation='relu'))
model.add(Dense(2048, kernel_initializer='normal', activation='relu'))
model.add(Dense(2048, kernel_initializer='normal', activation='relu'))
model.add(Dense(10, kernel_initializer='normal', activation='softmax'))

model.summary()




(60000, 28, 28)
197
(60000, 784)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 784)                 │         615,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │         803,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2048)                │       2,099,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2048)                │       4,196,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │          20,490 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,735,322 (29.51 MB)

 Trainable params: 7,735,322 (29.51 MB)

 Non-trainable params: 0 (0.00 B)

### Pergunta 1:
O elemento do índice 4000 do vetor de treinamento corresponde a qual classe?

Bag

### Pergunta 2:
Quantos são os parâmetros treináveis da VGG16?


33,638,218

### Pergunta 3:
Quantos são os parâmetros treináveis da segunda rede criada?

 7,735,322

### Pergunta 4:
Quantos são os pixels de cada amostra depois de transformadas para serem utilizadas na Vgg-Net?

1024

### Pergunta 5:
Caso você tente utilizar a VGG-Net com as imagens originais, sem redimensionamento, qual erro é obtido durante o treinamento?

Mensagem de erro informando que eram esperadas amostras de tamanho (32x32x3), mas recebeu amostras de tamanho (28x28x1).

### Pergunta 6:
Quantas são as classes presentes no problema?

10

### Pergunta 7:
Quantos são os pixels de cada amostra para serem utilizados na segunda rede?

784

### Pergunta 8:
O elemento do índice 4000 do vetor de teste corresponde à qual classe?





T-shirt/top

### Pergunta 9:
Se você utilizar a função de ativação relu na camada de saída da segunda rede, qual o valor obtido na loss?

Nan

### Pergunta 10:
Quantos bias são presentes na segunda rede?




5914

### Pergunta 11:
Quantas camadas do tipo ‘MaxPooling2D’ estão presentes na VGG-Net?

 5

### Pergunta 12:
Quantas camadas totalmente conectadas estão presentes na VGG-Net?

3

### Pergunta 13:
Execute a Vgg-Net com um batch de 128. Quantas iterações são necessárias para completar uma época?


469

### Pergunta 14:
Quantas camadas do tipo ‘Conv2D’ estão presentes na VGG-Net?


13

### Pergunta 15:
Quantos pesos existem conectando a penúltima e última camadas da segunda rede?

20480